# Gauss Map Visualization

This notebook walks through the process of generating a visualization of the Gauss map for an arbitrary parameterized surface in $\mathbb{R}^3$. This code was created from knowledge gained from my Differential Geometry class that used *Differential Geometry of Curves & Surfaces* by Manfredo do Carmo as a textbook [[1]](#REFERENCES).

A [Gauss Map](https://en.wikipedia.org/wiki/Gauss_Map) is a map from a surface in $\mathbb{R}^n$ to $S^{n-1} \subset \mathbb{R}^n$ where $S^{n-1}$ is the unit [sphere](https://en.wikipedia.org/wiki/N-sphere) of dimension $n-1$ [embedded](https://en.wikipedia.org/wiki/Embedding) in $\mathbb{R}^n$. This map sends points on the surface to points on the unit sphere corresponding to the unit vector normal to the surface at that point. This can be visualized as generating the normal vectors of the surface at various points and then shifting the "tail" of these vectors to the origin. That is exactly what this animation does.

## Libraries Used
We start by importing the various libraries that we will use. [Manim](https://www.manim.community/) is a math animation library originally written by Grant Sanderson of [3Blue1Brown](https://www.3blue1brown.com/) with a fork now maintained by the community [[2]](#REFERENCES). It allows us to setup a scene of objects, animate them, and render the output to a video.

Other important libraries used are [SymPy](https://www.sympy.org/) to calculate the partial derivatives and [vector product](https://en.wikipedia.org/wiki/Cross_product "cross product") used in the normal vector calculation, and [NumPy](https://numpy.org/) to numerically evaluate the normal vector at each point.

In [ ]:
from manim import (ThreeDScene, Surface, ThreeDAxes, VGroup, Vector, FadeIn, FadeOut, ApplyMethod, DEGREES, ORIGIN)
from sympy.abc import u, v
import sympy as sym
import numpy as np
from IPython.display import display, Math

## Typing

Typing annotations are used throughout the code. Typing in Python is optional and code will still run correctly even when incorrect types are used. I have however run the code through a static type checker so the type annotations used should be correct. The various types are defined here. Since the animation is limited to $\mathbb{R}^3$ then we define types using 3-tuples representing $x$, $y$, and $z$ coordinates. Additonally, we set the range value to a 2-tuple to represent the minimum and maximum values of the $u$ and $v$ coordinates.

In [ ]:
from typing import Tuple, Union
import numpy.typing as npt

Range = Tuple[float, float]

Matrices = Tuple[sym.Matrix, sym.Matrix, sym.Matrix]

# Expression representing x, y, and z coordinates
ParamExpr = Union[Tuple[sym.Expr, sym.Expr, sym.Expr], sym.Matrix]

# Vectorized function representing x, y, and z coordinates
ParamVectorize = Tuple[np.vectorize, np.vectorize, np.vectorize]

## Classes

### Surfaces
This manim object is a representation of the parameterized surface. The constructor takes in a vectorized parameterization of the surface that will be created elsewhere.

In [ ]:
class OriginalSurface(Surface):
    """A manim surface of the parameterization for the manimation."""
    def __init__(self, X: ParamVectorize, u_range: Range, v_range: Range,
                 r_max: int = 8, **kwargs) -> None:
        """Generates the parameterized surface for the manimation.

            Args:
                X: A tuple of vectorized functions that describes the
                    parameterized surface.
                u_range: A tuple containing the starting and ending u
                    coordinates.
                v_range: A tuple containing the staring and ending v
                    coordinates.
        """
        self.__r_max = r_max
        self.__X = X

        super().__init__(self.func, u_range=u_range, v_range=v_range, **kwargs)

    def func(self, u_i: float, v_i: float) -> npt.NDArray[np.float64]:
        """Evaluates the function at u and v to generate the surface."""
        X = _evaluate(self.__X, u_i, v_i)

        # limit graphs to sphere with radius r_max
        # a spherical boundary looks better than a cubic one
        r = np.linalg.norm(X)
        if r > self.__r_max:
            X = np.divide(np.multiply(self.__r_max, X),  r)
        return X

This manim object represents the subset of the unit sphere that the surface is sent to via the Gauss map. The unit sphere is the codomain of the Gauss map while this subset is its range. The vectorized parameterization of $\mathbf{x}$, and $\mathbf{n}$ of the unit normal vectors are generated elsewhere.

In [ ]:
class GaussMapSurface(Surface):
    """A manim surface of the Gauss map for the manimation."""
    def __init__(self, X: ParamVectorize, N: ParamVectorize, u_range: Range,
                 v_range: Range, **kwargs) -> None:
        """Generates the Gauss map surface for the manimation.

            Args:
                X: A tuple of vectorized functions that describes the orignal
                    surface.
                N: A tuple of vectorized functions that describes the normal
                    vectors of the original surface.
                u_range: A tuple containing the starting and ending u
                    coordinates.
                v_range: A tuple containing the starting and ending v
                    coordinates.
        """
        self.__N = N

        super().__init__(self.func, u_range=u_range, v_range=v_range, **kwargs)

    def func(self, u_i: float, v_i: float) -> npt.NDArray[np.float64]:
        """Evaluates the normal vectors at u and v to generate the surface."""
        N = _evaluate(self.__N, u_i, v_i)
        return _normalize(N)

### Vector Field
This manim object is a group of vector objects that represent the unit normal vectors at each point on the surface. The vectorized functions $\mathbf{x}$ of the parameterization, and $\mathbf{n}$ of the unit normal vectors are generated elsewhere.

In [ ]:
class VectorField(VGroup):
    """The normal vectors as a group of manim vectors for the manimation."""
    def __init__(self, X: ParamVectorize, N: ParamVectorize, u_range: Range,
                 v_range: Range, r_max: int = 8, amount: int = 20,
                 **kwargs) -> None:
        """Generates the normal vector field for the manimation.

            Args:

                X: A tuple of vectorized functions that describes the original
                    surface.
                N: A tuple of vectorized functions that describes the normal
                    vectors of the original surface.
                u_range: A tuple containing the starting and ending u
                    coordinates.
                v_range: A tuple containing the starting and ending v
                    coordinates.
                r_max: The maximum radius from the center where vectors are
                    generated.
                amount: The number of vectors to generate. They will be
                    dispersed evenly throughout the surface.
        """
        super().__init__(**kwargs)
        self.__r_max = r_max

        U = np.linspace(u_range[0], u_range[1], amount)
        V = np.linspace(v_range[0], v_range[1], amount)

        for u_i in U:
            for v_i in V:
                X_i = _evaluate(X, u_i, v_i)
                N_i = _evaluate(N, u_i, v_i)

                # ignore vectors that start outside a sphere of radius r_max
                r = np.linalg.norm(X_i)
                if r < self.__r_max:
                    self.add(self.__create_vector(X_i, N_i))

    @staticmethod
    def __create_vector(pos: npt.NDArray[np.float64],
                        N: npt.NDArray[np.float64]) -> Vector:
        """Evaluates the function at u and v to generate the vector field."""
        N = _normalize(N)
        
        # move vector away from the surface a bit so it does not clip through
        pos = pos + 0.1 * N

        # shade_in_3d prevents vectors and their tips from being visible when
        # they are behind 3d objects.
        vector = Vector(shade_in_3d=True, direction=N).shift(pos)
        vector.get_tip().set_shade_in_3d()
        return vector

### Scene

This scene is where all the objects are constructed and the animations are generated. Manim calls the construct function when rendering the scene so you can think of the construct method as the main method of this script.

In [ ]:
class GaussMap(ThreeDScene):
    """An animated scene that demonstrates the Gauss map of a surface."""
    def construct(self) -> None:
        """Creates the objects in the scene and plays the animation."""
        axes = ThreeDAxes()
        X_expr, u_range, v_range = _get_func()

        N_expr, X_u, X_v = _gauss(X_expr)
        display(Math(''.join(('\\mathbf{x} = ', sym.latex(X_expr)))))
        display(Math(''.join(('\\mathbf{x}_u = ', sym.latex((X_u[0], X_u[1],
                                                            X_u[2]))))))
        display(Math(''.join(('\\mathbf{x}_v = ', sym.latex((X_v[0], X_v[1],
                                                            X_v[2]))))))
        display(Math(''.join(('N = ', sym.latex((N_expr[0],
                                                 N_expr[1],
                                                 N_expr[2]))))))

        X = _sym_to_np(X_expr)
        N = _sym_to_np(N_expr)

        original_surface = OriginalSurface(X, u_range, v_range)
        gauss_map = GaussMapSurface(X, N, u_range, v_range)
        vector_field = VectorField(X, N, u_range, v_range)

        self.begin_ambient_camera_rotation(rate=0.1)
        self.set_camera_orientation(phi=45*DEGREES, theta=30*DEGREES)
        self.add(original_surface)
        self.wait(5)
        self.play(FadeIn(vector_field))
        self.wait(5)
        self.play(FadeOut(original_surface))
        self.remove(original_surface)
        animations = []
        for vector in vector_field:
            vec_mid = np.array(vector.get_vector()) / 2
            animations.append(ApplyMethod(vector.move_to, ORIGIN + vec_mid,
                                          run_time=0.2))
        self.play(*animations, run_time=3.0)
        self.wait(2)
        self.play(FadeIn(gauss_map))
        self.play(FadeOut(vector_field))
        self.remove(vector_field)
        self.wait(5)   

## Functions

This helper function is used in the various classes to evaluate a NumPy vectorized function at the parameterized $u$, $v$ coordinates.

In [ ]:
def _evaluate(V: ParamVectorize, u_i: float,
              v_i: float) -> npt.NDArray[np.float64]:
    """Evaluates the vectorized function at the given coordinates.

        Args:
            V: A tuple of vectorized functions to be evaluated.
            u_i: The u coordinate to evaluate at.
            v_i: The v coordinate to evaluate at.

        Returns:
            A numpy array of the evaluated values in x, y, and z coordinates.
    """
    x, y, z = V
    return np.array([x(u_i, v_i), y(u_i, v_i), z(u_i, v_i)])

This function converts expressions using $u$ and $v$ into NumPy vectorized functions that can be evaluated at specific coordinates.

In [ ]:
def _sym_to_np(X: ParamExpr) -> ParamVectorize:
    """Converts sympy expressions to numpy expressions.

    Args:
        X: A sequence of 3 sympy expressions to convert.

    Returns:
        A numpy array of numpy vectorized equations.
    """
    # iter needed for sympy matrix type checking
    x, y, z = iter(X)
    x_np = sym.lambdify([u, v], x, 'numpy')
    y_np = sym.lambdify([u, v], y, 'numpy')
    z_np = sym.lambdify([u, v], z, 'numpy')
    x_vec = np.vectorize(x_np)
    y_vec = np.vectorize(y_np)
    z_vec = np.vectorize(z_np)
    return x_vec, y_vec, z_vec   

In [ ]:
def _normalize(X: npt.NDArray[np.float64]) -> npt.NDArray[np.float64]:
    norm = np.linalg.norm(X)
    if norm == 0:
        norm = np.float64(1.0)
    return np.divide(X, norm)

This function is where the important math is done.

Let $U \subset \mathbb{R}^2$ be open and let $S \subset \mathbb{R}^3$ be a surface. $S$ is called a regular surface if for each $p \in S$ there exists a neighborhood $V$ in $\mathbb{R}^3$ and a map $\mathbf{x}: U \to V \cap S$ such that $\mathbf{x}$ is [differentiable](https://en.wikipedia.org/wiki/Differentiable_function), $\mathbf{x}$ is a [homeomorphism](https://en.wikipedia.org/wiki/Homeomorphism), and for each $(u, v) \in U$, the [differential](https://en.wikipedia.org/wiki/Pushforward_(differential) "pushforward") $d\mathbf{x}_{(u, v)}: \mathbb{R}^2 \to \mathbb{R}^3$ is [one-to-one](https://en.wikipedia.org/wiki/One-to-one_function "injective"). The map $\mathbf{x}$ is called a parameterization. See [[1, Def. 2-2.1, pp. 54-55]](#REFERENCES).

Let $V \subset S$ be an open set in $S$ and $N: V \to \mathbb{R}^3$ such that for each point $q \in V$, $N(q)$ is a unit normal vector at $q$. $N$ is a differentiable field of unit normal vectors on $V$ if this map is differentiable.

A regular surface $S \subset \mathbb{R}^3$ is orientable if there exists a differentiable field of unit normal vectors $N: S \to \mathbb{R}^3$ on the whole surface, $S$. See [[1, pp. 137-138]](#REFERENCES).

This means that for any two parameterizations, $\mathbf{x}: U \to S$ and $\mathbf{\tilde{x}}: \tilde{U} \to S$ and for all $(u, v) \in U$ and $(\tilde{u}, \tilde{v}) \in \tilde{U}$ that coincide then both

$$N(u, v) =\frac{\mathbf{x}_u(u, v) \times \mathbf{x}_v(u, v)}{||\mathbf{x}_u(u, v) \times \mathbf{x}_v(u, v)||}$$

$$N(\tilde{u}, \tilde{v}) =\frac{\mathbf{\tilde{x}}_{\tilde{u}}(\tilde{u}, \tilde{v}) \times \mathbf{x}_{\tilde{v}}(\tilde{u}, \tilde{v})}{||\mathbf{x}_{\tilde{u}}(\tilde{u}, \tilde{v}) \times \mathbf{x}_{\tilde{v}}(\tilde{u}, \tilde{v})||}$$

agree.


Let $S \subset \mathbb{R}^3$ be a orientable surface with orientation $N$. The map $N: S \to \mathbb{R}^3$ takes the values of $N$ to the unit sphere $S^2$. This map $N: S \to S^2$ is called the Gauss map of $S$. See [[1, Def. 3-2.1, p. 138]](#REFERENCES)

For our purposes we define our map as

$$N =\frac{\mathbf{x}_u \times \mathbf{x}_v}{||\mathbf{x}_u \times \mathbf{x}_v||}$$

In [ ]:
def _gauss(X: ParamExpr) -> Matrices:
    """Calculates the normal vector and partial derivatives of X.

    Args:
        X: A parametrized equation in terms of u and v where the
            elements describe the x, y, and z coordinates in that order.

    Returns:
        A tuple (N, X_u, X_v) where N is the parameterized equation of the
        normal vector in terms of u and v, X_u is the parameterized
        equation of the paritial derivative of X with respect to u in terms
        of u and v, and X_v is the parameterized equations of the partial
        derivative of X with respect to v in terms of u and v.
    """
    # iter needed for sympy Matrix type checking
    x, y, z = iter(X)
    x_u = sym.diff(x, u)
    x_v = sym.diff(x, v)
    y_u = sym.diff(y, u)
    y_v = sym.diff(y, v)
    z_u = sym.diff(z, u)
    z_v = sym.diff(z, v)
    X_u = sym.Matrix([x_u, y_u, z_u])
    X_v = sym.Matrix([x_v, y_v, z_v])
    N = X_u.cross(X_v)

    return N, X_u, X_v

This function converts strings into floats and SymPy expressions. **Note:** No input validation is done.

In [ ]:
def _get_func() -> Tuple[ParamExpr, Range, Range]:
    u_range = (float(sym.sympify(u_min)), float(sym.sympify(u_max)))
    v_range = (float(sym.sympify(v_min)), float(sym.sympify(v_max)))
    X = (sym.sympify(x), sym.sympify(y), sym.sympify(z))
    return X, u_range, v_range

## Rendering

We use cell magic `%%manim` to run the Manim command-line utility from the jupyter notebook. We pass in the scene name `GaussMap` to render it. We pass `-v WARNING` to remove unneccessary output and `-ql` to set the render quality to low so that it renders faster. Other options include `-qm` for medium, and `-qh` for high quality.

Here we define the maximum and minimum values for the $u$ and $v$ coordinates and the parameterized functions in terms of $u$ and $v$. [$\LaTeX$](https://www.latex-project.org/) of the partial derivative and normal vector output is shown as well as the rendered video. 

Please try your own functions some cool demos are found below.

### [Cone](https://mathworld.wolfram.com/Cone.html)

In [ ]:
%%manim -v WARNING -ql GaussMap

# Change me!
u_min = '0.01' # Remove singular point at (0, 0, 0)
u_max = '2'
v_min = '0'
v_max = '2*pi'
x = 'u*cos(v)'
y = 'u*sin(v)'
z = 'u'

### [One-Sheeted Hyperboloid](https://mathworld.wolfram.com/One-SheetedHyperboloid.html)

In [ ]:
%%manim -v WARNING -ql GaussMap

# Change me!
u_min = '-2*pi'
u_max = '2*pi'
v_min = '0'
v_max = '2*pi'
x = 'cosh(u)*cos(v)'
y = 'cosh(u)*sin(v)'
z = 'sinh(u)'

### [Catenoid](https://mathworld.wolfram.com/Catenoid.html)

In [ ]:
%%manim -v WARNING -ql GaussMap

# Change me!
u_min = '-pi'
u_max = 'pi'
v_min = '-2'
v_max = '2'
x = '2*cosh((1/2)*v)*cos(u)'
y = '2*cosh((1/2)*v)*sin(u)'
z = 'v'

### [Ring Torus](https://mathworld.wolfram.com/RingTorus.html)

In [ ]:
%%manim -v WARNING -ql GaussMap

# R = 3, r = 1

# Change me!
u_min = '0'
u_max = '2*pi'
v_min = '0'
v_max = '2*pi'
x = '(3 + 1*cos(u))*cos(v)'
y = '(3 + 1*cos(u))*sin(v)'
z = '1*sin(u)'

### [Paraboloid](https://mathworld.wolfram.com/Paraboloid.html)

In [ ]:
%%manim -v WARNING -ql GaussMap

# Change me!
u_min = '0.001'
u_max = '2'
v_min = '0'
v_max = '2*pi'
x = 'u*cos(v)'
y = 'u*sin(v)'
z = 'u^2'

### [Hyperbolic Paraboloid](https://mathworld.wolfram.com/HyperbolicParaboloid.html)

In [ ]:
%%manim -v WARNING -ql GaussMap

# Change me!
u_min = '-2'
u_max = '2'
v_min = '-2'
v_max = '2'
x = 'u'
y = 'v'
z = 'v^2-u^2'

### [Monkey Saddle](https://mathworld.wolfram.com/MonkeySaddle.html)

In [ ]:
%%manim -v WARNING -ql GaussMap

# Change me!
u_min = '-3'
u_max = '3'
v_min = '-3'
v_max = '3'
x = 'u'
y = 'v'
z = 'u^3-3*v^2*u'

## REFERENCES

[1] M. P. DO CARMO, *Differential Geometry of Curves and Surfaces*, 2nd ed., Dover, Mineola, NY, 2016.

[2] MANIM COMMUNITY DEVELOPERS AND G. SANDERSON, *Manim software*, 2022, https://www.manim.community (accessed 2022/09/22). Version 0.16.0.post0.